## Extract image features

In [1]:
import os
import torch
import torchvision.transforms as transforms
from torchvision.models import resnet18
from PIL import Image

# Load pre-trained ResNet model and remove the top fully connected layer
base_model = resnet18(pretrained=True)
base_model = torch.nn.Sequential(*(list(base_model.children())[:-1]))

batch = "batch1"

# Define image preprocessing
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]), # the mean and std come from imagenet dataset
])

# Define dataset folder path
trainset = f'dataset_white/{batch}_train'
valset = f'dataset_white/{batch}_val'
testset = f'dataset_white/{batch}_test'

# Create an empty list to store features
train_features = []
val_features = []
test_features = []

# Iterate through all image files in the dataset folder
for filename in sorted(os.listdir(trainset)):
    if filename.endswith('.jpg') or filename.endswith('.jpeg') or filename.endswith('.png'):
        # Construct image path
        image_path = os.path.join(trainset, filename)

        # Load image
        img = Image.open(image_path)

        # Preprocess image and add batch dimension
        img_tensor = preprocess(img).unsqueeze(0)

        # Set model to evaluation mode
        base_model.eval()

        # Use ResNet model for feature extraction
        with torch.no_grad():
            features = base_model(img_tensor)

        # Append features to the list
        train_features.append(features.squeeze())


# Iterate through all image files in the dataset folder
for filename in sorted(os.listdir(valset)):
    if filename.endswith('.jpg') or filename.endswith('.jpeg') or filename.endswith('.png'):
        # Construct image path
        image_path = os.path.join(valset, filename)

        # Load image
        img = Image.open(image_path)

        # Preprocess image and add batch dimension
        img_tensor = preprocess(img).unsqueeze(0)

        # Set model to evaluation mode
        base_model.eval()

        # Use ResNet model for feature extraction
        with torch.no_grad():
            features = base_model(img_tensor)

        # Append features to the list
        val_features.append(features.squeeze())

# Iterate through all image files in the dataset folder
for filename in sorted(os.listdir(testset)):
    if filename.endswith('.jpg') or filename.endswith('.jpeg') or filename.endswith('.png'):
        # Construct image path
        image_path = os.path.join(testset, filename)

        # Load image
        img = Image.open(image_path)

        # Preprocess image and add batch dimension
        img_tensor = preprocess(img).unsqueeze(0)

        # Set model to evaluation mode
        base_model.eval()

        # Use ResNet model for feature extraction
        with torch.no_grad():
            features = base_model(img_tensor)

        # Append features to the list
        test_features.append(features.squeeze())


c:\Users\user\anaconda3\envs\avocado\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\user\anaconda3\envs\avocado\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


## Read labals 

In [2]:
import os
import pandas as pd

# Load label.csv file
label_df = pd.read_csv(f'label/{batch}label.csv')
labelname = "Firmness"

# Initialize lists to store labels
train_labels = []
val_labels = []
test_labels = []

# Iterate through all image files in the dataset folder
for filename in sorted(os.listdir(trainset)):
    if filename.endswith('.jpg'):
        # Extract filename without extension
        filename_without_extension = os.path.splitext(filename.replace("original_", ""))[0]

        # Extract label from label.csv based on filename
        label = label_df[label_df["Image_name"] == filename_without_extension][labelname].iloc[0]
        train_labels.append(label)

for filename in sorted(os.listdir(valset)):
    if filename.endswith('.jpg'):
        # Extract filename without extension
        filename_without_extension = os.path.splitext(filename)[0]
        
        
        # Extract label from label.csv based on filename
        label = label_df[label_df["Image_name"] == filename_without_extension][labelname].iloc[0]
        val_labels.append(label)

for filename in sorted(os.listdir(testset)):
    if filename.endswith('.jpg'):
        # Extract filename without extension
        filename_without_extension = os.path.splitext(filename)[0]
        
        # Extract label from label.csv based on filename
        label = label_df[label_df["Image_name"] == filename_without_extension][labelname].iloc[0]
        test_labels.append(label)

train_labels = [torch.tensor([float(train_labels[i])]) for i in range(len(train_labels))]
val_label = [torch.tensor([float(val_labels[i])]) for i in range(len(val_labels))]
test_label = [torch.tensor([float(test_labels[i])]) for i in range(len(test_labels))]

train_features_tensor = torch.stack(train_features)
train_labels_tensor = torch.tensor(train_labels)
val_features_tensor = torch.stack(val_features)
val_labels_tensor = torch.tensor(val_labels)
test_features_tensor = torch.stack(test_features)
test_labels_tensor = torch.tensor(test_labels)


## Random forest

In [3]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import GridSearchCV
from tqdm import tqdm
import numpy as np

# Define the random forest model
# 定义随机森林模型
model = RandomForestRegressor(random_state=42)

# Define the parameter grid for hyperparameter tuning
# 定义参数空间
param_grid = {
    'n_estimators': [100, 200, 300],  # Number of trees in the forest
    'max_depth': [None, 10, 20, 30],  # Maximum depth of the tree
    'min_samples_split': [2, 5, 10],  # Minimum number of samples required to split an internal node
    'min_samples_leaf': [1, 2, 4]  # Minimum number of samples required to be at a leaf node
}

# Create a GridSearchCV object with parallel computation
# 创建 GridSearchCV 对象，并设置并行计算
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)

# Fit the grid search to the data
# 拟合网格搜索模型
grid_search.fit(train_features_tensor, train_labels_tensor)

# Output the best parameter combination
# 输出最佳参数组合
print("Best Parameters:", grid_search.best_params_)

# Train the model with the best parameters
# 使用最佳参数训练模型
best_model = grid_search.best_estimator_
best_model.fit(train_features_tensor, train_labels_tensor)

# Make predictions on the training and validation sets
# 在训练集和验证集上进行预测
train_predictions = best_model.predict(train_features_tensor)
val_predictions = best_model.predict(val_features_tensor)

# Calculate mean squared error and R^2 score
# 计算均方误差和 R^2 分数
train_loss = mean_squared_error(train_labels_tensor, train_predictions)
val_loss = mean_squared_error(val_labels_tensor, val_predictions)
train_r2 = r2_score(train_labels_tensor, train_predictions)
val_r2 = r2_score(val_labels_tensor, val_predictions)

# Output performance metrics on the training and validation sets
# 输出训练和验证集上的性能指标
print(f'Train Loss: {train_loss:.4f}, Validation Loss: {val_loss:.4f}')
print(f'Train R-squared: {train_r2:.4f}, Validation R-squared: {val_r2:.4f}')

# Make predictions on the test set
# 在测试集上进行预测
test_predictions = best_model.predict(test_features_tensor)
mse = mean_squared_error(test_labels_tensor, test_predictions)
r_squared = r2_score(test_labels_tensor, test_predictions)

# Output predictions and performance metrics on the test set
# 输出测试集上的预测结果和性能指标
print("Predictions:", test_predictions)
print("True Labels:", test_labels_tensor.tolist())
print(f'Test RMSE: {np.sqrt(mse):.4f}')
print(f'Test R-squared: {r_squared:.4f}')

## SVR

In [ ]:
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import GridSearchCV
import numpy as np

# Define the SVR model
# 定义 SVR 模型
svr = SVR()

# Define the parameter search space
# 定义参数搜索范围
param_grid = {
    'kernel': ['linear', 'poly', 'rbf'],  # Kernel types
    'C': [0.01, 0.1, 1, 10, 100],        # Regularization parameter
    'epsilon': [0.01, 0.1, 1, 10]        # Epsilon in the epsilon-SVR model
}

# Create the GridSearchCV object with parallel computation
# 创建 GridSearchCV 对象，并设置并行计算
grid_search = GridSearchCV(estimator=svr, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)

# Fit the grid search to the data
# 拟合网格搜索模型
grid_search.fit(train_features_tensor, train_labels_tensor)

# Output the best parameter combination
# 输出最佳参数组合
print("Best Parameters:", grid_search.best_params_)

# Train the model with the best parameters
# 使用最佳参数训练模型
best_model = grid_search.best_estimator_
best_model.fit(train_features_tensor, train_labels_tensor)

# Make predictions on the training and validation sets
# 在训练集和验证集上进行预测
train_predictions = best_model.predict(train_features_tensor)
val_predictions = best_model.predict(val_features_tensor)

# Calculate mean squared error and R^2 score
# 计算均方误差和 R^2 分数
train_loss = mean_squared_error(train_labels_tensor, train_predictions)
val_loss = mean_squared_error(val_labels_tensor, val_predictions)
train_r2 = r2_score(train_labels_tensor, train_predictions)
val_r2 = r2_score(val_labels_tensor, val_predictions)

# Output performance metrics on the training and validation sets
# 输出训练和验证集上的性能指标
print(f'Train Loss: {train_loss:.4f}, Validation Loss: {val_loss:.4f}')
print(f'Train R-squared: {train_r2:.4f}, Validation R-squared: {val_r2:.4f}')

# Make predictions on the test set
# 在测试集上进行预测
test_predictions = best_model.predict(test_features_tensor.numpy())
test_labels = test_labels_tensor.numpy()

# Calculate mean squared error and R^2 score on the test set
# 计算测试集上的均方误差和 R^2 分数
mse = mean_squared_error(test_labels, test_predictions)
r_squared = r2_score(test_labels, test_predictions)

# Output predictions and performance metrics on the test set
# 输出测试集上的预测结果和性能指标
print("Predictions:", test_predictions)
print("True Labels:", test_labels)
print(f'Test RMSE: {np.sqrt(mse):.4f}')
print(f'Test R-squared: {r_squared:.4f}')

Best Parameters: {'C': 100, 'epsilon': 1, 'kernel': 'poly'}
Train Loss: 5.3380, Validation Loss: 48.1908
Train R-squared: 0.9927, Validation R-squared: 0.9337
Predictions: [ 94.33963579  72.00758087 108.1429415   79.67156906 102.49441478
  82.49212959  94.45033592  88.04539692  67.1464648   56.11540785
  68.81761468  34.87813469  23.11210297  40.74211461  43.21345708
  25.38798723  48.92473034  44.40180247  35.9659079   69.74911261
  19.20977327  18.74982472  19.0770864   19.19478266  16.23212522
  11.9606066   24.61869313  14.02637904  18.59422095  17.8670587
  16.39806388  14.63508806  11.36055709  14.85089256  16.28190623
  16.37381075  16.86944242  14.8118663   15.46615728  15.42793197
  17.08013809  13.32591866  21.16734793  11.21901377  11.3273566
  18.30495644  16.78834238  10.73494637   2.46863228  23.29641483
  11.45083424  12.44299423  12.40831651  12.66784094  15.41648394
  14.44667607  18.52455654  19.5579224   12.4719516   17.58990389
  12.3178813    8.14811292  13.7587926

In [ ]:
# from sklearn.svm import SVR
# from sklearn.metrics import mean_squared_error, r2_score
# from sklearn.model_selection import GridSearchCV
# import numpy as np



# params = {
#     'C': 100,
#     'epsilon': 0.01,
#     'kernel': 'rbf',
# }


# model = SVR(**params)

# model.fit(train_features_tensor, train_labels_tensor)

# # 在训练集和验证集上进行预测
# train_predictions = model.predict(train_features_tensor)
# val_predictions = model.predict(val_features_tensor)

# # 计算均方误差和 R^2 分数
# train_loss = mean_squared_error(train_labels_tensor, train_predictions)
# val_loss = mean_squared_error(val_labels_tensor, val_predictions)
# train_r2 = r2_score(train_labels_tensor, train_predictions)
# val_r2 = r2_score(val_labels_tensor, val_predictions)

# # 输出训练和验证集上的性能指标
# print(f'Train Loss: {train_loss:.4f}, Validation Loss: {val_loss:.4f}')
# print(f'Train R-squared: {train_r2:.4f}, Validation R-squared: {val_r2:.4f}')

# # 在测试集上进行预测
# test_predictions = model.predict(test_features_tensor.numpy())
# test_labels = test_labels_tensor.numpy()

# # 计算测试集上的均方误差和 R^2 分数
# mse = mean_squared_error(test_labels, test_predictions)
# r_squared = r2_score(test_labels, test_predictions)

# # 输出测试集上的预测结果和性能指标
# print("Predictions:", test_predictions)
# print("True Labels:", test_labels)
# print(f'Test RMSE: {np.sqrt(mse):.4f}')
# print(f'Test R-squared: {r_squared:.4f}')


